In [2]:
import folium

In [3]:
import pandas as pd

In [4]:
unternehmen = pd.read_csv("Unternehmen.csv", delimiter=";")
personen = pd.read_csv("Personen.csv", delimiter=";")
gegenstaende = pd.read_csv("Gegenstände.csv", delimiter=";")

map = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

In [5]:
for _, row in unternehmen.iterrows():
    # Popup-Text erstellen
    popup_html = f"""
    <b>Unternehmen:</b> {row['Name']}<br>
    <b>Standort:</b> {row['Standort']}<br>
    <b>Identifier:</b> <a href="{row['Identifier']}" target="_blank">Website</a><br>
    """
    
    # Verknüpfte Personen hinzufügen (aus Gegenstandstabelle)
    related_items = gegenstaende[gegenstaende['Unternehmen ID'] == row['ID']]
    if not related_items.empty:
        popup_html += "<b>Verknüpfte Personen:</b><ul>"
        for pid in related_items['Hersteller:innen ID'].values:
            for person_id in map(int, pid.split(',')):
                person_row = personen[personen['ID'] == person_id]
                if not person_row.empty:
                    person = person_row.iloc[0]
                    popup_html += f'<li><a href="{person["Identifier"]}" target="_blank">{person["Name"]}</a></li>'
        popup_html += "</ul>"

    # Marker hinzufügen
    folium.Marker(
        location=[row['Breitengrad'], row['Längengrad']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=row['Name']
    ).add_to(map)

In [6]:
map